In [1]:
import os
import json
import numpy as np
import pickle
import random

import data_loader.task_configs as task_configs
from future.modules import ptl2classes

Loading faiss with AVX2 support.


In [2]:
for dataset in ['pawsx', 'xnli']: # marc, mldoc, pawsx, xnli
    raw_dataset = task_configs.task2dataset[dataset]()
    num_bucket = 40
    
    if dataset == 'marc':
        shots = [1, 2, 4, 8]
        langs = ['chinese', 'french', 'german', 'japanese', 'spanish']
    elif dataset == 'pawsx':
        shots = [1, 2, 4, 8]
        langs = ['chinese', 'french', 'german', 'japanese', 'korean', 'spanish']
    elif dataset == 'xnli':
        shots = [1, 2, 4, 8]
        langs = ['arabic', 'bulgarian', 'chinese', 'french', 'german', 'greek', 'hindi', 'russian', 'spanish', 'swahili', 'thai', 'turkish', 'urdu', 'vietnamese']
    else:
        raise ValueError
    
    for shot in shots:
        for lang in langs:
            lang_dataset = raw_dataset.get_language_data(language=lang)
            val_egs = lang_dataset.val_egs
            
            label_list = []
            for eg in val_egs:
                label_list.append(eg.label)
            label_list = np.array(label_list)
            
            selected_items = {}
            for l in lang_dataset.label_list:
                np.random.seed(42)
                selected_items[l] = np.random.choice(np.where(label_list==l)[0], size=shot*num_bucket, replace=False)
            
            all_selected_idx = []
            
            for bucket in range(num_bucket):
                pkl_data = {'eg_names': [], 'actual_egs': {}}
                
                for l in lang_dataset.label_list:
                    selected_idx = selected_items[l][shot*bucket:shot*(bucket+1)].tolist()
                    pkl_data['eg_names'] += selected_idx
                    pkl_data['actual_egs'][l] = []
                    for idx in selected_idx:
                        pkl_data['actual_egs'][l].append(val_egs[idx])
                    all_selected_idx += pkl_data['eg_names']
                    pkl_data['eg_names'] = [str(i) for i in pkl_data['eg_names']]
                
                save_filename = './sampled_infos/sampled_data/{}/{}-shots/{}/{}-th/'.format(dataset, shot, lang, bucket)
                os.makedirs(save_filename, exist_ok = True)
                save_filename += 'sampled.pkl'
                with open(save_filename, 'wb') as f:
                    pickle.dump(pkl_data, f)
                    
            pkl_data = {}
            for l in lang_dataset.label_list:
                left_dev_idx = set(np.where(label_list==l)[0]) - set(all_selected_idx)
                pkl_data[l] = []
                for idx in left_dev_idx:
                    pkl_data[l].append(val_egs[idx])
            
            save_filename = './sampled_infos/sampled_data/{}/{}-shots/{}/left,dev.pkl'.format(dataset, shot, lang)
            with open(save_filename, 'wb') as f:
                pickle.dump(pkl_data, f)

[INFO]: skip de trn: not such file
./data/download/pawsx/dev-de.tsv 2000
./data/download/pawsx/test-de.tsv 2000
./data/download/pawsx/train-en.tsv 49401
./data/download/pawsx/dev-en.tsv 2000
./data/download/pawsx/test-en.tsv 2000
[INFO]: skip es trn: not such file
./data/download/pawsx/dev-es.tsv 2000
./data/download/pawsx/test-es.tsv 2000
[INFO]: skip fr trn: not such file
./data/download/pawsx/dev-fr.tsv 2000
./data/download/pawsx/test-fr.tsv 2000
[INFO]: skip ja trn: not such file
./data/download/pawsx/dev-ja.tsv 2000
./data/download/pawsx/test-ja.tsv 2000
[INFO]: skip ko trn: not such file
./data/download/pawsx/dev-ko.tsv 2000
./data/download/pawsx/test-ko.tsv 2000
[INFO]: skip zh trn: not such file
./data/download/pawsx/dev-zh.tsv 2000
./data/download/pawsx/test-zh.tsv 2000
2490 ./data/download/xnli/dev-ar.tsv
5010 ./data/download/xnli/test-ar.tsv
2490 ./data/download/xnli/dev-bg.tsv
5010 ./data/download/xnli/test-bg.tsv
2490 ./data/download/xnli/dev-de.tsv
5010 ./data/download/xn

In [ ]:
dataset = 'xnli'
shot = 1
lang = 'chinese'
bucket = 15


save_filename = './sampled_infos/sampled_data/{}/{}-shots/{}/{}-th/sampled.pkl'.format(dataset, shot, lang, bucket)
save_filename = './sampled_infos/sampled_data/{}/{}-shots/{}/left,dev.pkl'.format(dataset, shot, lang, bucket)

with open(save_filename, 'rb') as f:
    pkl = pickle.load(f)

In [ ]:
for dataset in ['panx', 'udpos']:
    raw_dataset = task_configs.task2dataset[dataset]()
    num_bucket = 40
    
    if dataset == 'panx':
        shots = [1, 2, 4]
        langs = ['af', 'ar', 'bg', 'bn', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'he', 'hi', 'hu', 'id', 'it', 'ja', 'jv', 'ka', 'kk', 'ko', 'ml', 'mr', 'ms', 'my', 'nl', 'pt', 'ru', 'sw', 'ta', 'te', 'th', 'tl', 'tr', 'ur', 'vi', 'yo', 'zh']
    elif dataset == 'udpos':
        shots = [1, 2, 4]
        langs = ['af', 'ar', 'bg', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'he', 'hi', 'hu', 'id', 'it', 'ja', 'ko', 'mr', 'nl', 'pt', 'ru', 'ta', 'te', 'tr', 'ur', 'vi', 'zh']
    else:
        raise ValueError